In [1]:
import re
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
#from tqdm import tqdm_notebook
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.utils import shuffle

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## 1. 훈련 캡션 데이터 전처리

In [2]:
annotation = pd.read_csv('../Segmentation/COCO/annotations/captions_train2017.csv', index_col=0)
annotation.head()

,image_id,id,caption
0,391895,770337,A man with a red helmet on a small moped on a ...
1,391895,771687,Man riding a motor bike on a dirt road on the ...
2,391895,772707,A man riding on the back of a motorcycle.
3,391895,776154,A dirt path with a young person on a motor bik...
4,391895,781998,A man in a red shirt and a red hat is on a mot...


In [3]:
all_captions = []
all_img_name_vector = []

In [4]:
for annot in tqdm(annotation.iterrows()):
    caption = '<start> ' + annot[1]['caption'] + ' <end>'
    image_id = annot[1]['image_id']
    full_img_image_path = '../Segmentation/COCO/segTrain2017/' + '{0:0>12}.png'.format(image_id)

    all_img_name_vector.append(full_img_image_path)
    all_captions.append(caption)

591753it [00:56, 10565.61it/s]


In [5]:
all_captions, all_img_name_vector= shuffle(all_captions,
                                           all_img_name_vector,
                                           random_state=1)

In [6]:
all_captions

['<start> A man holding a device while walking down a sidewalk. <end>',
 '<start> A train pulling into a train station with no people on the platform.  <end>',
 '<start> two elephant walking in a field with a handler beside them  <end>',
 '<start> An elephant outside near a fence with its trunk in its mouth. <end>',
 '<start> There is a firetruck and fireman by a fire hydrant with a hose. <end>',
 '<start> A house eats long grass in the field. <end>',
 '<start> A group of sheep in dirt field next to fence and barn. <end>',
 '<start> A woman that is standing with a little girl. <end>',
 '<start> an image of a fridge outside on the side walk <end>',
 '<start> a close up of a person eating pizza  <end>',
 '<start> a kid holding a bat ready to play in the field <end>',
 '<start> The train has stopped and the doors are open. <end>',
 '<start> an older man standing at ta busy corner holding signs <end>',
 '<start> Three zebras being observed in their natural habitat. <end>',
 '<start> A grou

In [7]:
all_img_name_vector

['../Segmentation/COCO/segTrain2017/000000393649.png',
 '../Segmentation/COCO/segTrain2017/000000304552.png',
 '../Segmentation/COCO/segTrain2017/000000522454.png',
 '../Segmentation/COCO/segTrain2017/000000191464.png',
 '../Segmentation/COCO/segTrain2017/000000187469.png',
 '../Segmentation/COCO/segTrain2017/000000516827.png',
 '../Segmentation/COCO/segTrain2017/000000328036.png',
 '../Segmentation/COCO/segTrain2017/000000397929.png',
 '../Segmentation/COCO/segTrain2017/000000051025.png',
 '../Segmentation/COCO/segTrain2017/000000265037.png',
 '../Segmentation/COCO/segTrain2017/000000034574.png',
 '../Segmentation/COCO/segTrain2017/000000055121.png',
 '../Segmentation/COCO/segTrain2017/000000535737.png',
 '../Segmentation/COCO/segTrain2017/000000198706.png',
 '../Segmentation/COCO/segTrain2017/000000208511.png',
 '../Segmentation/COCO/segTrain2017/000000537543.png',
 '../Segmentation/COCO/segTrain2017/000000427348.png',
 '../Segmentation/COCO/segTrain2017/000000164189.png',
 '../Segme

In [8]:
print('Number of all_img_name_vector :', len(all_img_name_vector))
print('Number of all_captions :', len(all_captions))

Number of all_img_name_vector : 591753
Number of all_captions : 591753


In [9]:
np.save(open('../Segmentation/COCO/train_input.npy', 'wb'), all_img_name_vector)
np.save(open('../Segmentation/COCO/train_label.npy', 'wb'), all_captions)

## 2. 검증 캡션 데이터 전처리

In [10]:
annotation = pd.read_csv('../Segmentation/COCO/annotations/captions_val2017.csv', index_col=0)
annotation.head()

,image_id,id,caption
0,397133,370509,A man is in a kitchen making pizzas.
1,397133,370584,Man in apron standing on front of oven with pa...
2,397133,372252,A baker is working in the kitchen rolling dough.
3,397133,372765,A person standing by a stove in a kitchen.
4,397133,375891,A table with pies being made and a person stan...


In [11]:
all_captions = []
all_img_name_vector = []

In [12]:
for annot in tqdm(annotation.iterrows()):
    caption = '<start> ' + annot[1]['caption'] + ' <end>'
    image_id = annot[1]['image_id']
    full_img_image_path = '../Segmentation/COCO/segVal2017/' + '{0:0>12}.png'.format(image_id)
    
    all_img_name_vector.append(full_img_image_path)
    all_captions.append(caption)

25014it [00:02, 10244.92it/s]


In [13]:
all_captions, all_img_name_vector= shuffle(all_captions,
                                           all_img_name_vector,
                                           random_state=1)

In [14]:
all_captions

['<start> A toddler sits on a toilet while brushing his teeth. <end>',
 '<start> A beautiful zebra standing in a green field. <end>',
 "<start> A cat playing with it's reflection in a mirror. <end>",
 '<start> A street with many motor bikes parked together. <end>',
 '<start> A small bathroom has an army green toilet. <end>',
 '<start> A dining table has a large pizza and wine glasses. <end>',
 '<start> a single person riding a big wave with a surfboard  <end>',
 '<start> a canal with lots of boats and buildings on either side <end>',
 '<start> a close of up a clock that makes the moon look small next to it <end>',
 '<start> A man wearing a hat while standing in front of a bathroom mirror. <end>',
 '<start> A white plate topped with food next to a cup of coffee. <end>',
 '<start> A small boat in water beside a sea airplane. <end>',
 '<start> Two giraffes that are standing in the grass. <end>',
 '<start> A large airplane flying in the blue sky. <end>',
 '<start> A pit bull dog on a messy

In [15]:
all_img_name_vector

['../Segmentation/COCO/segVal2017/000000368212.png',
 '../Segmentation/COCO/segVal2017/000000537053.png',
 '../Segmentation/COCO/segVal2017/000000411665.png',
 '../Segmentation/COCO/segVal2017/000000524108.png',
 '../Segmentation/COCO/segVal2017/000000180792.png',
 '../Segmentation/COCO/segVal2017/000000047112.png',
 '../Segmentation/COCO/segVal2017/000000297595.png',
 '../Segmentation/COCO/segVal2017/000000463849.png',
 '../Segmentation/COCO/segVal2017/000000155051.png',
 '../Segmentation/COCO/segVal2017/000000202228.png',
 '../Segmentation/COCO/segVal2017/000000256518.png',
 '../Segmentation/COCO/segVal2017/000000205401.png',
 '../Segmentation/COCO/segVal2017/000000285788.png',
 '../Segmentation/COCO/segVal2017/000000379453.png',
 '../Segmentation/COCO/segVal2017/000000022192.png',
 '../Segmentation/COCO/segVal2017/000000306136.png',
 '../Segmentation/COCO/segVal2017/000000366711.png',
 '../Segmentation/COCO/segVal2017/000000179898.png',
 '../Segmentation/COCO/segVal2017/000000261061

In [16]:
print('Number of all_img_name_vector :', len(all_img_name_vector))
print('Number of all_captions :', len(all_captions))

Number of all_img_name_vector : 25014
Number of all_captions : 25014


In [17]:
np.save(open('../Segmentation/COCO/val_input.npy', 'wb'), all_img_name_vector)
np.save(open('../Segmentation/COCO/val_label.npy', 'wb'), all_captions)